In [22]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
# sys.path.insert(0, "..")

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [23]:
from app.creators import universe
from app.functions import maths

In [24]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

In [25]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'built_by': 'type of object that can build the building'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'changes': 'stats changed by the owner'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in `universal time units` that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"}]

## Adding a building to a pop, for testing. 

On a newly created population, adding a building to it so that I can test it. 
[refer to the ontology](../../docs/img/Exodestiny_Ontology.png)

In [27]:
farm = buildings["buildings"][0]
farm["objid"] = maths.uuid()
farm["label"] = "building"
farm

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block',
 'objid': '7099158193479',
 'label': 'building'}

In [28]:
popid = "6722043178391"
systemid = "1747270367183"
username = "BillmanLocal2"

In [29]:
edge = {"node1": farm["objid"], "node2": popid, "label": "owned_by"}

In [30]:
data = {"nodes": [farm], "edges": [edge]}

In [31]:
c.upload_data(username, data)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000298B715FD68>


In [32]:
building_query = f"""
g.V().has('objid','{popid}').as('pop')
    .in('owned_by').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['6722043178391'], 'name': ['Velty Rie']},
   {'objid': ['7099158193479'],
    'name': ['farmland'],
    'changes': ['{wealth: -0.01}'],
    'augments_resource': ['{vegitable_foodstuff: 10}'],
    'planet_requirements': ['{isHabitable: true}'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'render_type': ['block']}]}]

In [33]:
building_query = f"""g.V().has('objid','{systemid}').as('location')
    .in('inhabits').as('pop')
    .in('owned_by').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.add_query(building_query)
c.run_queries()


existing_buildings = []
for iter, item in enumerate(c.res[building_query]):
    build = c.clean_node(item["objects"][2])
    owner = c.clean_node(item["objects"][1])
    build.update({"owner": owner["objid"]})
    existing_buildings.append(build)
existing_buildings

[{'objid': '7099158193479',
  'name': 'farmland',
  'changes': '{wealth: -0.01}',
  'augments_resource': '{vegitable_foodstuff: 10}',
  'planet_requirements': '{isHabitable: true}',
  'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
  'render_type': 'block',
  'id': '7099158193479',
  'owner': '6722043178391'}]

In [59]:
# query = f"""
# g.V().has('objid','{farm['objid']}').drop()
# """

# c.run_query(query)
# c.res

[]

In [81]:
buildings

[{'objid': '6193583697156',
  'name': 'farmland',
  'changes': '{wealth: -0.01}',
  'augments_resource': '{vegitable_foodstuff: 10}',
  'planet_requirements': '{isHabitable: true}',
  'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
  'render_type': 'block',
  'id': '6193583697156'}]